## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the Personalized Ranking solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/recommendations-batch.html). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": <S3 input path>}},
    jobOutput = 
       {"s3DataDestination": {"path": <S3 output path>}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for Personalized Ranking, it looks like:


```JSON
{"userId": "891", "itemList": ["27", "886", "101"]}
{"userId": "445", "itemList": ["527", "55", "901"]}
{"userId": "71", "itemList": ["27", "351", "101"]}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"891","itemList":["27","886","101"]},"output":{"recommendedItems":["27","101","886"],"scores":[0.48421,0.28133,0.23446]}}
{"input":{"userId":"445","itemList":["527","55","901"]},"output":{"recommendedItems":["901","527","55"],"scores":[0.46972,0.31011,0.22017]}}
{"input":{"userId":"71","itemList":["29","351","199"]},"output":{"recommendedItems":["351","29","199"],"scores":[0.68937,0.24829,0.06232]}}

```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.



To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [ ]:
#retrieves previously stored variables 
%store -r 

In [ ]:
personalize = boto3.client('personalize')
# personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
# personalize_events = boto3.client(service_name='personalize-events')

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [ ]:
# We will use the same users from before
print (users)
# Write the file to disk
json_input_filename = "json_input.json"
with open(json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '", "itemList":'+json.dumps(unranked_product_ids)+'}\n')

In [ ]:
# Showcase the input file:
!cat $json_input_filename

Upload the file to S3 and save the path as a variable for later.

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = workshop_rerank_solution_version_arn,
    jobName = "POC-Batch-Inference-Job-PersonalizedRanking_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [ ]:
%%time

current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Let's create a helper function:

In [ ]:
def get_item_name_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['name'].values[0]
    return item_name

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_item_name_from_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

In [ ]:
%store workshop_dataset_group_arn
%store region
%store role_name
%store s3_access_policy_arn

# Variables required for Optional Notebook 1 - Batch Inference
%store users
%store unranked_product_ids
%store bucket_name
%store workshop_rerank_solution_version_arn
%store item_metadata_df

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in [`Retail_06_Clean_Up.ipynb`](Retail_06_Clean_Up.ipynb).

You can choose to head to any of the optional notebooks in this folder to continue experimenting with Amazon Personalize.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: Run optional notebooks BEFORE you run the clean-up notebook.
</div>